In [7]:
import requests
from dotenv import load_dotenv
import os
import json
import base64   
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import string
import tempfile
from datetime import datetime, timedelta


In [8]:
# For any api we can try using the "+" email trick to get more API keys

genius_client_id = "wZZ2RWc5mqp-5Pbz2W1rQJWE8LQ3pFBrb1Hw5_AOqgybq28mt7kjdjcG4zktCNbO"
genius_client_secret = "PefqBJHor_muDgTutGlaXXaxmzsI7TQCps9FQ3FwkUTT0WJIT3s0A5YA9mnFbfp_-CBhQF7b0omgE8kaM3dJ3w"
genius_access_token = "NUHHVpwnmbDYUYw8Padu0gQeHvYN4OsKYtE2MKNUpBUI6yR-xZXKY6S5NvCnFbiP"

lastfm_api_key = "97d5a64d5ba4a8bc580b752ceff3b87f"
lastfm_secret = "35175090bd61f6f16ac607bd26e5b1de"

In [9]:
base_url = 'http://ws.audioscrobbler.com/2.0/'

def lastfm_get(payload):
    headers = {'user-agent': 'DataCollectorBot'}
    payload['api_key'] = lastfm_api_key
    payload['format'] = 'json'
    response = requests.get(base_url, headers=headers, params=payload)
    return response.json()

# Function for each endpoint
def get_user_info(user):
    payload = {'method': 'user.getinfo', 'user': user}
    return lastfm_get(payload)

def get_top_albums(user):
    payload = {'method': 'user.gettopalbums', 'user': user}
    return lastfm_get(payload)

def get_top_artists(user):
    payload = {'method': 'user.gettopartists', 'user': user}
    return lastfm_get(payload)

def get_top_tracks(user):
    payload = {'method': 'user.gettoptracks', 'user': user}
    return lastfm_get(payload)



In [10]:
def list_to_df(data_list, columns):
    if data_list:
        df = pd.DataFrame(data_list, columns=columns)
        return df
    else:
        return pd.DataFrame(columns=columns)
    
def get_top_albums_df(user):
    result = get_top_albums(user)
    albums = []
    for item in result['topalbums']['album']:
        albums.append({
            'Rank': item['@attr']['rank'],
            'Album Name': item['name'],
            'Artist': item['artist']['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(albums, ['Rank', 'Album Name', 'Artist', 'Play Count'])

def get_top_artists_df(user):
    result = get_top_artists(user)
    artists = []
    for item in result['topartists']['artist']:
        artists.append({
            'Rank': item['@attr']['rank'],
            'Artist': item['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(artists, ['Rank', 'Artist', 'Play Count'])


def get_top_tracks_df(user):
    result = get_top_tracks(user)
    tracks = []
    for item in result['toptracks']['track']:
        tracks.append({
            'Rank': item['@attr']['rank'],
            'Track Name': item['name'],
            'Artist': item['artist']['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(tracks, ['Rank', 'Track Name', 'Artist', 'Play Count'])

In [15]:
def get_combined_top_data(user, top_n=10):
    # Retrieve top albums, artists, and tracks dataframes limited to top_n entries
    top_albums_df = get_top_albums_df(user).head(top_n)
    top_artists_df = get_top_artists_df(user).head(top_n)
    top_tracks_df = get_top_tracks_df(user).head(top_n)
    
    # Make sure that the 'Rank' column is of the same data type across all DataFrames before concatenating
    top_albums_df['Rank'] = top_albums_df['Rank'].astype(int)
    top_artists_df['Rank'] = top_artists_df['Rank'].astype(int)
    top_tracks_df['Rank'] = top_tracks_df['Rank'].astype(int)

    # Align DataFrames by 'Rank' and horizontally concatenate
    combined_df = pd.concat([top_albums_df.set_index('Rank'),
                             top_artists_df.set_index('Rank'),
                             top_tracks_df.set_index('Rank')],
                            axis=1,
                            keys=['Albums', 'Artists', 'Tracks'])
    
    # Reset the index so 'Rank' becomes a column again
    combined_df.reset_index(inplace=True)

    return combined_df



df2 = get_combined_top_data("Bans77")
df2

Rank                     Albums                             \
                       Album Name          Artist Play Count   
0    1        Discography, Vol. 1      VELVETEARS       2831   
1    2                   3 Ghosts      VELVETEARS       1477   
2    3             Думала ты одна     Даша Каплан       1455   
3    4  мои (твои) тёмные желания            ooes       1210   
4    5        Discography, Vol. 2      VELVETEARS       1009   
5    6             Pins & Needles      VELVETEARS        921   
6    7             Мне не страшно            ooes        748   
7    8                         22  Erika Lundmoen        689   
8    9                   Immunity          Clairo        688   
9   10                 Hikikomori      VELVETEARS        683   

               Artists                                 Tracks  \
                Artist Play Count                  Track Name   
0           VELVETEARS      10911              Думала ты одна   
1                 ooes       2723              Sincere Regret   
2               Clairo       2251                       Minor   
3        Alexis Munroe       1972                Central Park   
4              Grechka       1519                     nothing   
5          Даша Каплан       1456         Million Dollar Song   
6     Maggie Lindemann       1259                      больше   
7               Zheani       1068                    Imma Lie   
8  Side of Despondency       1053            Will I Ever Care   
9              DSPRITE       1020  We All Wanna Die Sometimes   

                                   
                Artist Play Count  
0          Даша Каплан       1456  
1  Side of Despondency        357  
2        Gracie Abrams        345  
3           VELVETEARS        291  
4           VELVETEARS        265  
5           VELVETEARS        255  
6                 ooes        254  
7           VELVETEARS        253  
8           VELVETEARS        251  
9           VELVETEARS        243